1. k均值算法收敛得到的解一定是局部最优吗？一定是全局最优吗？ A. 是 B. 不是

---

2. 如果设定的$K$值比数据中实际的类别要少，会造成什么情况？是否有办法在不知道数据包含几类的情况下，选出合适的$K$值？

---

3. k均值算法的结果及其依赖于初始中心的选取，从而对随机种子非常敏感。试构造一个数据集，在$K=3$的情况下，存在至少两组 K 均值算法可能收敛到的聚类中心，且数据的分类不同。如果换成k-means++算法，在该数据集上收敛的结果是否唯一？

---

4. 除了k-means++以外，k均值算法还有一种改进，称为二分k均值。该算法首先将所有数据看作一类。接下来每次迭代中，找到类内部所有点距离之和
$$S(C_k)=\frac{1}{2}\sum_{x_i,x_j \in C_k}d(x_i,x_j)$$
最大的类，再将该类分成两个使得类内距离之和下降最多的子类。如此循环，直到类的数量达到预先指定的$K$为止。实现该算法，并在14.2节中的数据集上测试。

---

5. 设计一种新的距离度量函数，实现基于该度量函数的k-means和k-means++，并在14.2节中的数据集上测试。

---

6. 查阅相关文献，学习并实现DBSCAN聚类算法，并和k-means、k-means++对比。